In [1]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.robust.scale import huber
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
import locale
import geopandas
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)


# df_indicadores_pob = pd.read_parquet(f'../data/processed/df_indicadores_pob.parquet')
# df_final_pob = pd.read_parquet('../data/processed/df_final_pob.parquet')
# df_final_pob_melt = pd.read_parquet('../data/processed/df_final_pob_melt.parquet')
# df_final_pob_melt_PC = pd.read_parquet('../data/processed/df_final_pob_melt_PC.parquet')
# CCAA_CO = pd.read_parquet('../data/processed/CCAA_CO.parquet')
# MUNI_CO = pd.read_parquet('../data/processed/MUNI_CO.parquet')
# PROV_CO = pd.read_parquet('../data/processed/PROV_CO.parquet')



In [2]:
df_final_pob=pd.read_parquet('../data/main_processed/df_final_pob.parquet')

df_indicadores_pob=pd.read_parquet('../data/main_processed/df_indicadores_pob.parquet')

df_indicadores_pob_pivot=pd.read_parquet('../data/main_processed/df_indicadores_pob_pivot.parquet')

df_final_pob_melt=pd.read_parquet('../data/main_processed/df_final_pob_melt.parquet')

df_final_pob_melt_PC=pd.read_parquet('../data/main_processed/df_final_pob_melt_PC.parquet')

CCAA_CO = pd.read_parquet('../data/main_processed/CCAA_CO.parquet')
MUNI_CO = pd.read_parquet('../data/main_processed/MUNI_CO.parquet')
PROV_CO = pd.read_parquet('../data/main_processed/PROV_CO.parquet')

df_final_pob_dropdown=pd.read_parquet('../data/main_processed/df_final_pob_dropdown.parquet')

df_final_pob_dropdown_c=pd.read_parquet('../data/main_processed/df_final_pob_dropdown_c.parquet')

df_final_pob_poblaciontext=pd.read_parquet('../data/main_processed/df_final_pob_poblaciontext.parquet')




In [48]:
CCAA_types='Andalucía'
municipio_types= 'Sevilla'
cohorte='1.000-2.000'
partida_de_coste_types='Abastecimiento de agua potable'

In [46]:
df = df_final_pob
df1=df.iloc[[0]]

In [47]:
df1

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,Abastecimiento de agua potable,"Abastos, mercados, lonjas",Acceso a los núcleos de población,Actividades funerarias,Alcantarillado,Alumbrado público,Biblioteca pública,Cementerio,Comercio ambulante,Conservación y rehabilitación de la edificación,"Conservación, mantenimiento y vigilancia de los edificios destinados a educación",Cooperar con las Administraciones educativas en la obtención de solares,Equipamientos culturales,Evacuación y tratamiento de aguas residuales,Ferias,Información y promoción de la actividad turística de interés y ámbito local,Infraestructura viaria y otros equipamientos de titularidad de la entidad local,Instalaciones de ocupación del tiempo libre,Instalaciones deportivas,Instalaciones deportivas de uso público,Limpieza viaria,Medio ambiente urbano,Medio ambiente urbano: Gestión de los residuos sólidos urbanos,Medio ambiente urbano: Parques y jardines públicos,Necesidad social y atención inmediata a personas en riesgo de exclusión social,Parque público,Participar en la vigilancia del cumplimiento de la escolaridad obligatoria,Pavimentación de las vías públicas,Policía local,Prevención y extinción de incendios,Promoción de la cultura,Promoción de la participación de los ciudadanos en las T.I.C.,Promoción del deporte,Promoción y gestión de la vivienda de protección pública,Protección civil,...,PC_Biblioteca pública,PC_Cementerio,PC_Comercio ambulante,PC_Conservación y rehabilitación de la edificación,"PC_Conservación, mantenimiento y vigilancia de los edificios destinados a educación",PC_Cooperar con las Administraciones educativas en la obtención de solares,PC_Equipamientos culturales,PC_Evacuación y tratamiento de aguas residuales,PC_Ferias,PC_Información y promoción de la actividad turística de interés y ámbito local,PC_Infraestructura viaria y otros equipamientos de titularidad de la entidad local,PC_Instalaciones de ocupación del tiempo libre,PC_Instalaciones deportivas,PC_Instalaciones deportivas de uso público,PC_Limpieza viaria,PC_Medio ambiente urbano,PC_Medio ambiente urbano: Gestión de los residuos sólidos urbanos,PC_Medio ambiente urbano: Parques y jardines públicos,PC_Necesidad social y atención inmediata a personas en riesgo de exclusión social,PC_Parque público,PC_Participar en la vigilancia del cumplimiento de la escolaridad obligatoria,PC_Pavimentación de las vías públicas,PC_Policía local,PC_Prevención y extinción de incendios,PC_Promoción de la cultura,PC_Promoción de la participación de los ciudadanos en las T.I.C.,PC_Promoción del deporte,PC_Promoción y gestión de la vivienda de protección pública,PC_Protección civil,"PC_Protección contra contaminación acústica, lumínica y atmosférica",PC_Protección de la salubridad pública,PC_Protección y gestión del Patrimonio histórico,PC_Recogida de residuos,PC_Transporte colectivo urbano de viajeros,PC_Tratamiento de residuos,"PC_Tráfico, estacionamiento de vehículos y movilidad","PC_Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística",PC_TOTAL,cohorte_pob,codigo_geo
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,282928.39,0.0,16467.5,4615.09,98943.76,179881.08,8877.06,23513.28,18018.62,6210.62,59795.42,0.0,1085.9,80972.55,146163.96,0.0,35439.53,6249.48,23581.92,29393.67,57423.21,5818.66,6078.05,15279.77,20755.44,26665.37,197.52,28064.65,13979.51,0.0,15306.22,6325.31,23014.77,6498.97,0.0,...,7.187903,19.039093,14.589976,5.028842,48.417344,0.0,0.879271,65.564818,118.351385,0.0,28.695976,5.060308,19.094672,23.800543,46.496526,4.711466,4.921498,12.372283,16.806024,21.591393,0.159935,22.724413,11.319441,0.0,12.3937,5.121709,18.635441,5.262324,0.0,0.0,4.189951,1.342227,0.0,0.0,0.0,3.449522,1.842899,1020.982721,1.000-2.000,4001


In [49]:



%%timeit

df = df_final_pob_melt_PC[['Nombre Ente Principal' , 'cohorte_pob' , 'Descripción' , 'coste_efectivo_PC']].loc[
                (df_final_pob_melt_PC['cohorte_pob'] == cohorte)  & ( df_final_pob_melt_PC['Descripción'] == partida_de_coste_types)].sort_values(by='coste_efectivo_PC' , ascending=False)

4.11 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
df2=df_final_pob_melt_PC[['Nombre Ente Principal' , 'cohorte_pob' , 'Descripción' , 'coste_efectivo_PC']]

In [53]:
%%timeit

df = df2.loc[(df2['cohorte_pob'] == cohorte)  & ( df2['Descripción'] == partida_de_coste_types)].sort_values(by='coste_efectivo_PC' , ascending=False)

2.52 ms ± 89.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
df_final_pob_melt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79997 entries, 0 to 209366
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   CCAA                   79997 non-null  category
 1   Provincia              79997 non-null  category
 2   Tipo Ente Principal    79997 non-null  category
 3   Nombre Ente Principal  79997 non-null  category
 4   Código Ente Principal  79997 non-null  category
 5   cohorte_pob            79997 non-null  category
 6   codigo_geo             79997 non-null  category
 7   Descripción            79997 non-null  category
 8   coste_efectivo         79997 non-null  float64 
dtypes: category(8), float64(1)
memory usage: 2.6 MB


In [21]:
%%timeit

df_cohorte.loc[df_cohorte['Nombre Ente Principal'] == municipio_types , 'cohorte_pob']\
                          .unique().to_list()[0]

981 µs ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
df_cohorte.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4869 entries, 0 to 4868
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Nombre Ente Principal  4869 non-null   category
 1   cohorte_pob            4869 non-null   category
dtypes: category(2)
memory usage: 250.7 KB


In [54]:
df_final_pob_melt

,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,cohorte_pob,codigo_geo,Descripción,coste_efectivo
0,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,1.000-2.000,4001,Abastecimiento de agua potable,282928.39
1,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,1.000-2.000,4002,Abastecimiento de agua potable,77227.64
2,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,20.000-50.000,4003,Abastecimiento de agua potable,1255574.16
3,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,0-1.000,4004,Abastecimiento de agua potable,60828.11
4,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,0-1.000,4005,Abastecimiento de agua potable,157266.48
6,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,0-1.000,4007,Abastecimiento de agua potable,61945.05
7,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,0-1.000,4008,Abastecimiento de agua potable,50007.27
8,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,0-1.000,4009,Abastecimiento de agua potable,36579.39
9,Andalucía,Almería,Ayuntamiento,Alhabia,01-04-010-AA-000,0-1.000,4010,Abastecimiento de agua potable,21280.85
...,...,...,...,...,...,...,...,...,...


In [60]:
df_final_pob_melt_PC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79997 entries, 0 to 209366
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   CCAA                   79997 non-null  category
 1   Provincia              79997 non-null  category
 2   Nombre Ente Principal  79997 non-null  category
 3   cohorte_pob            79997 non-null  category
 4   codigo_geo             79997 non-null  category
 5   Descripción            79997 non-null  category
 6   coste_efectivo_PC      79997 non-null  float64 
 7   Población 2018         79997 non-null  float64 
dtypes: category(6), float64(2)
memory usage: 2.8 MB


In [59]:
df_final_pob_melt_PC=df_final_pob_melt_PC.drop(columns=['Tipo Ente Principal','Código Ente Principal'])